In [1]:
import findspark

In [2]:
findspark.init('/home/pushya/spark-2.1.0-bin-hadoop2.7')

In [3]:
import pyspark

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [9]:
from pyspark.ml.regression import LinearRegression

In [13]:
# Load training data
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [15]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [25]:
data.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

In [26]:
for item in data.head(2)[1]:
    print (item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [27]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [28]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [31]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent'],outputCol='features'
                           
                           
                           
                           
                           )

In [36]:
output = assembler.transform(data)

In [40]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [41]:
final_data = output.select('features','Yearly Amount Spent')

In [43]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [44]:
train_data,test_data= final_data.randomSplit([0.7,0.3])

In [46]:
train_data.count()

366

In [47]:
test_data.count()

134

In [53]:
train_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [51]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                134|
|   mean|  496.1080265849369|
| stddev|   79.2537371690658|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



In [56]:
lr = LinearRegression(
    
    labelCol='Yearly Amount Spent'
    
)

In [59]:
lr_model= lr.fit(train_data)

In [61]:
test_results = lr_model.evaluate(test_data)

In [62]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|2.216893335571512...|
|2.046363078989088...|
|-1.02318153949454...|
|5.002220859751105...|
|-5.68434188608080...|
|-4.54747350886464...|
|3.069544618483633...|
|2.330580173293128...|
|1.705302565824240...|
|5.456968210637569...|
|1.250555214937776...|
|1.818989403545856...|
|-1.81898940354585...|
|7.389644451905042...|
|2.955857780762016...|
|6.366462912410498...|
|3.524291969370097...|
|                 0.0|
|2.330580173293128...|
|9.094947017729282...|
+--------------------+
only showing top 20 rows



In [64]:
test_results.rootMeanSquaredError

3.1579476825410435e-12

In [65]:
test_results.r2

1.0

In [66]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [68]:
unlabeled_data= test_data.select('features')

In [69]:
unlabeled_data.printSchema()

root
 |-- features: vector (nullable = true)



In [71]:
predictions = lr_model.transform(unlabeled_data)

In [73]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 282.4712457199123|
|[30.8364326747734...|467.50190042698756|
|[31.0472221394875...| 392.4973991890224|
|[31.0662181616375...|448.93329320766935|
|[31.3091926408918...| 432.7207178399342|
|[31.4459724827577...|  484.876964935129|
|[31.4474464941278...|418.60274209522095|
|[31.5147378578019...| 489.8124879964591|
|[31.5171218025062...|  275.918420650384|
|[31.5741380228732...| 544.4092721605814|
|[31.5761319713222...| 541.2265839893271|
|[31.6253601348306...| 376.3369007569224|
|[31.6548096756927...|475.26342372755033|
|[31.6610498227460...| 416.3583535799001|
|[31.7207699002873...|   538.77493347802|
|[31.8164283341993...|   501.12249150365|
|[31.8279790554652...|  440.002747546938|
|[31.8293464559211...|  385.152337987975|
|[31.8745516945853...|392.28524424626517|
|[31.9120759292006...| 387.5347163057068|
+--------------------+------------

In [74]:
test_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[30.4925366965402...|  282.4712457199145|
|[30.8364326747734...|  467.5019004269896|
|[31.0472221394875...|  392.4973991890214|
|[31.0662181616375...| 448.93329320767435|
|[31.3091926408918...|  432.7207178399336|
|[31.4459724827577...| 484.87696493512857|
|[31.4474464941278...|   418.602742095224|
|[31.5147378578019...|  489.8124879964614|
|[31.5171218025062...|  275.9184206503857|
|[31.5741380228732...|  544.4092721605869|
|[31.5761319713222...|  541.2265839893283|
|[31.6253601348306...|  376.3369007569242|
|[31.6548096756927...|  475.2634237275485|
|[31.6610498227460...| 416.35835357990084|
|[31.7207699002873...|   538.774933478023|
|[31.8164283341993...| 501.12249150365636|
|[31.8279790554652...|  440.0027475469415|
|[31.8293464559211...|   385.152337987975|
|[31.8745516945853...|  392.2852442462675|
|[31.9120759292006...|  387.5347163057077|
+----------